# Feature Extraction usuing Wav2vec base model

In [1]:
import pandas as pd
import os
import os
import torch
import soundfile as sf
import numpy as np
from torchvision.datasets import DatasetFolder

data_path ='/Feature_extraction_wav2vec/SEP_Fluency/'
dataset = DatasetFolder(data_path, loader=torch.load, extensions=[".wav"])
classes = dataset.classes
class_to_idx = {classes[i]: i for i in range(len(classes))}



audio_data = []
for c in class_to_idx:
    class_path = os.path.join(data_path, c)
    for file in os.listdir(class_path):
        if file.endswith(".wav"):
            audio_data.append([os.path.join(class_path, file), c])

df = pd.DataFrame(audio_data, columns=["file", "label"])


In [2]:
df

,file,label
0,/Feature_extraction_wav2vec/SEP_Fluency/Block\...,Block
1,/Feature_extraction_wav2vec/SEP_Fluency/Block\...,Block
2,/Feature_extraction_wav2vec/SEP_Fluency/Block\...,Block
3,/Feature_extraction_wav2vec/SEP_Fluency/Block\...,Block
4,/Feature_extraction_wav2vec/SEP_Fluency/Block\...,Block
...,...,...
35485,/Feature_extraction_wav2vec/SEP_Fluency/WordRe...,WordRep
35486,/Feature_extraction_wav2vec/SEP_Fluency/WordRe...,WordRep
35487,/Feature_extraction_wav2vec/SEP_Fluency/WordRe...,WordRep
35488,/Feature_extraction_wav2vec/SEP_Fluency/WordRe...,WordRep


In [3]:
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2Model

#feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")


Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2Model: ['lm_head.weight', 'lm_head.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:

features = []
labels = []

for i, row in df.iterrows():
    audio, sample_rate = sf.read(row["file"])
    input_values = processor(audio,sampling_rate=sample_rate, return_tensors="pt").input_values
    with torch.no_grad():
        feature = model(input_values).last_hidden_state
    features.append(feature)
    labels.append(row["label"])
    torch.save(feature, f"/Feature_extraction_wav2vec/FeaturesByLargModel/{row['label']}_{i}.pt")
    

KeyboardInterrupt: 

# Save the fetures into sngle tensor

In [1]:
import torch
import os

# Set the path to your saved features
features_path = "/Feature_extraction_wav2vec/Last_Features/features/"

# Define a list to hold the features and labels
features_list = []
labels_list = []

# Loop over each saved feature file
for filename in os.listdir(features_path):
    # Load the feature tensor from disk
    features = torch.load(os.path.join(features_path, filename))

    # Get the label from the filename
    label = filename.split("_")[0]
    

    # Append the feature and label to their respective lists
    features_list.append(features)
    labels_list.append(label)


# Features to numpy

In [2]:
import numpy as np
for i, feature in enumerate(features_list):
    if feature.size()[1] != 149:
        print(i)
        del features_list[i]

3569
3634
3722
5237
5407
5701
6620
7233
7569
7728
7877
7930
8003
8042
8048
8184
8351
8409
8422
8450
8787
8798
8810
8848
8877
8939
8955
9237
9593
9606
9674
9725
9732
9757
9954
12104
12392
12397
18358
18359
18452
20346
22132
24178
24185
24377
27762
29871
30469
30586


In [8]:
# Concatenate the tensors in smaller batches
feature_chunks1 = torch.cat(features_list[34011:35543], dim=0)
The_Features = feature_chunks1.numpy()
The_Features = np.concatenate([The_Features, feature_chunks1.numpy()], axis=0)
The_Features.shape

(4532, 149, 768)

In [24]:
# The_Features = features_tensor.numpy()
The_Features.shape

# Save the features tensor to disk
np.save("/Feature_extraction_wav2vec/allFFF/features2.npy",The_Features)

(15000, 149, 768)

In [1]:
# read the features in the memory
import numpy as np
featuree = np.load("/Feature_extraction_wav2vec/allFFF/concatenatedfeatures.npy", mmap_mode='r')
concatenated_features = np.concatenate((featuree, The_Features), axis=0)
np.save("/Feature_extraction_wav2vec/allFFF/features.npy",concatenated_features)


# Labels to numpy

In [ ]:
# Create a dictionary to map letters to numbers
letter_to_number = {'Block': 0, 'Fluent': 1, 'Interjection': 2, 'Prolongation': 3, 'SoundRep': 4, 'WordRep': 5}

# Loop through the list and replace each letter with its corresponding number
for i in range(len(labels_list)):
    labels_list[i] = letter_to_number[labels_list[i]]

#print(labels_list)

labels_tensor = torch.Tensor(labels_list)
labels_tensor.shape
The_Labels = labels_tensor.numpy()

In [2]:
# read the features in the memory
import numpy as np
featuree = np.load("/Feature_extraction_wav2vec/allFFF/features.npy", mmap_mode='r')
labeel = np.load("/Feature_extraction_wav2vec/allFFF/labels.npy", mmap_mode='r')
print(featuree.shape)
print(labeel.shape)

(35532, 149, 768)
(35594,)


In [4]:
indices_to_remove = [35543,35542,35541,35540,35539,35538,35537,35536,35535,35534,3553,35532]
labeel = np.delete(labeel, indices_to_remove)
np.save("/Feature_extraction_wav2vec/allFFF/labels2.npy",labeel)

print(featuree.shape)
print(labeel.shape)

(35532, 149, 768)
(35532,)


# Split the Data

In [ ]:
from sklearn.model_selection import train_test_split
# split the data into training, validation, and test sets
X_train, X_test, y_train, y_test = train_test_split(featuree, labeel, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# save the split datasets to files
np.save("/Feature_extraction_wav2vec/data_split13G/train_features.npy", X_train)
np.save("/Feature_extraction_wav2vec/data_split13G/train_labels.npy", y_train)
np.save("/Feature_extraction_wav2vec/data_split13G/val_features.npy", X_val)
np.save("/Feature_extraction_wav2vec/data_split13G/val_labels.npy", y_val)
np.save("/Feature_extraction_wav2vec/data_split13G/test_features.npy", X_test)
np.save("/Feature_extraction_wav2vec/data_split13G/test_labels.npy", y_test)